In [14]:
from modified_shipping import *

In [15]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0
    @staticmethod
    def _make_bic_code(owner_code, serial):
        return iso6346.create(owner_code=owner_code, serial=str(serial).zfill(6), category='R')

    def __init__(self, owner_code, contents, celsius):
        super().__init__(owner_code, contents)
        if celsius > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too hot!")
        self.celsius = celsius

Try it out:

In [16]:
r3 = RefrigeratedShippingContainer.create_with_items('ESC', ['broccoli', 'cauliflower', 'carrots'], celsius=2.0)

In [17]:
r3

In [18]:
r3.contents

['broccoli', 'cauliflower', 'carrots']

In [19]:
r3.celsius

2.0

In [20]:
r3.bic

'ESCU0013388'

Unfortunately, this design also ignores the constraint defined by the MAX_CELSIUS class attribute:

In [21]:
r3.celsius = 12

The poing of MAX_CELSUIUS is that hte temperature in a refrigerated container should never rise above that value.  Setting the temperature to 12 violates a class invariant, that will need to be prevented.  To accomplish this Python allows for properties, which is a superior alternative to getters and setters.  Properties allow getters and setters to be exposed as seemingly regular attributes, permitting a graceful update in capabilities.

In the RefrigeratedShippingContainer class, we will rename celsius to _celsius to indicate that it is no longer to be considered part of the public interface and also define a new method celsius() which will retrieve the attribue.  The method will be decorated with the built-in property decorator:

In [23]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0
    @staticmethod
    def _make_bic_code(owner_code, serial):
        return iso6346.create(owner_code=owner_code, serial=str(serial).zfill(6), category='R')

    def __init__(self, owner_code, contents, celsius):
        super().__init__(owner_code, contents)
        if celsius > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too hot!")
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

In [24]:
r4 = RefrigeratedShippingContainer.create_with_items('YML', ['fish'], celsius=-18.0)

In [25]:
r4.celsius

-18.0

The above showed that the @property decorator has converted our celsius() method into something that behaves like an attribute when accessed.  Undertand that @property can be used to transform getter methods so they can be called as if they were attributes.

An attempt to assign celsius will provide an AttributeError informing that the attribute cannot be set:

In [26]:
r4.celsius = -5.0

AttributeError: can't set attribute

To make assignemt to properties work a setter must be defined